In [1]:
from random import randint
from PIL import Image
import numpy as np
import pickle
import pycuda.driver as cuda
import pycuda.autoinit
import uff
import tensorrt as trt
import ctypes
from matplotlib import pyplot as plt
from six.moves import cPickle
import logging


In [ ]:
# Initializes logger for debugging purposes
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")
TRT_LOGGER = trt.Logger(trt.Logger.INFO)
curr_parent_dir =  os.path.dirname(os.getcwd())

In [2]:
## information about data needed to parse model and build engine
class ModelData(object):
    MODEL_FILE = curr_parent_dir + "/models/bvlc_googlenet/weights.caffemodel"
    DEPLOY_FILE = curr_parent_dir + "/models/bvlc_googlenet/deploy.prototxt"
    INPUT_SHAPE = (3, 224, 224) # image dimensions - nchw
    OUTPUT_NAME = "prob" # output layer name
    DTYPE = trt.float32 # type of optimization FP32, FP16.
    BATCH_SIZE = 500; # maximum batch size of engine to use for inference
    WORKSPACE_SIZE = 1 << 30 # The maximum GPU temporary memory which the ICudaEngine can use at execution time.
    ENGINE_NAME = "fp32.engine"

In [5]:

# parses model to trt
with trt.Builder(TRT_LOGGER) as builder, builder.create_network() as network, trt.CaffeParser() as parser:
    # builder argums for optimization
    builder.max_batch_size = ModelData.BATCH_SIZE
    builder.max_workspace_size = ModelData.WORKSPACE_SIZE  
    # Load the Caffe model and parse it in order to populate the TensorRT network.
    # This function returns an object that we can query to find tensors by name.
    model_tensors = parser.parse(deploy=ModelData.DEPLOY_FILE, model=ModelData.MODEL_FILE, network=network, dtype=ModelData.DTYPE)
    # For Caffe, we need to manually mark the output of the network.
    # Since we know the name of the output tensor, we can find it in model_tensors.
    print(model_tensors)
    network.mark_output(model_tensors.find(ModelData.OUTPUT_NAME))

    # builds engine
    with builder.build_cuda_engine(network) as engine:
        with open(ModelData.ENGINE_NAME, "wb") as f:
            f.write(engine.serialize())
